# Análisis de sentimiento con RNNs y word *embeddings*

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

## Entrenamiento con *embeddings*

In [2]:
MAX_WORDS = 20000   # Número máximo de palabras para cargar y ENTRENAR el embedding

# Cargar datos de IMDB
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=MAX_WORDS, start_char=None)

print('Shape de los datos X de entrenamiento: ', x_train.shape)
print('Shape de los datos Y de entrenamiento: ', y_train.shape)
print('Shape de los datos X de validación: ', x_val.shape)
print('Shape de los datos Y de validación: ', y_val.shape)

Shape de los datos X de entrenamiento:  (25000,)
Shape de los datos Y de entrenamiento:  (25000,)
Shape de los datos X de validación:  (25000,)
Shape de los datos Y de validación:  (25000,)


In [6]:
# Inspección de un ejemplo
# x_train[0]
# y_train[0]

In [9]:
# Obtener el diccionario de los reviews
word_index = keras.datasets.imdb.get_word_index(path="imdb_word_index.json")
# word_index

In [10]:
# Vamos a preprocesar los datos, en secuencias de tamaño fijo de largo 200
MAX_SEQ_LENGTH = 200
x_train_padded = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=MAX_SEQ_LENGTH)
x_val_padded = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=MAX_SEQ_LENGTH)

print('Shape de los datos X de entrenamiento: ', x_train_padded.shape)
print('Shape de los datos X de validación: ', x_val_padded.shape)

Shape de los datos X de entrenamiento:  (25000, 200)
Shape de los datos X de validación:  (25000, 200)


In [13]:
# Definición del modelo de sentimiento
WORD_REPR_DIM = 128 # Dimensión del vector que representa cada palabra

sentiment_model = keras.models.Sequential([
    # Capa de embeddings para trasladar cada número en las secuencias a un vector 
    # de representación de cada palabra
    keras.layers.Embedding(MAX_WORDS, WORD_REPR_DIM, input_length=200),
    # Unidades LSTM para aprender los patrones en las secuencias que determinan un
    # comentario positivo o negativo
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(64),
    # Unidad de salida, clasifica si el comentario es positivo o negativo
    keras.layers.Dense(1, activation="sigmoid")
])

sentiment_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 200, 64)           49408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,642,497
Trainable params: 2,642,497
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compilación y entrenamiento del modelo
sentiment_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = sentiment_model.fit(x = x_train_padded, y = y_train, \
    batch_size=32, epochs=2, \
    validation_data = (x_val_padded, y_val))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.3982 - accuracy: 0.8194 - val_loss: 0.3048 - val_accuracy: 0.8695
Epoch 2/2
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.2145 - accuracy: 0.9190 - val_loss: 0.3207 - val_accuracy: 0.8663


In [15]:
def sentiment_review(review_str, sentiment_model): 
    # Obtener la secuencia de números de palabras
    # 'this movie is awesome very exciting and romantic' -> [11, 17, 6, 1187, 52, 1124, 2, 728]
    review_seq = [word_index[token] for token in review_str.split()]
    # Hacer padding de la secuencia
    # [11, 17, 6, 1187, 52, 1124, 2, 728] -> [0, 0, ..., 0, 11, 17, 6, 1187, 52, 1124, 2, 728]
    review_seq_padded = keras.preprocessing.sequence.pad_sequences([review_seq], maxlen=MAX_SEQ_LENGTH)
    # Obtener el sentimiento con el modelo y devolverlo
    sentiment = sentiment_model(review_seq_padded)
    return sentiment.numpy().flatten()[0]


In [18]:
# Ahora vamos a utilizarlo para un ejemplo propio =P
# Después de un preprocesamiento de signos de puntuación y exclamación 
# podríamos tener algo así: 
own_reviews = ['this movie is awesome very exciting', \
    'awful movie']


In [19]:
# Mostramos el punteo de sentimiento de los reviews
for review in own_reviews: 
    print('Review: "', review, '" -> ', sentiment_review(review, sentiment_model))

Review: " this movie is awesome very exciting " ->  0.6356226
Review: " awful movie " ->  0.43795732


---
## Entrenamiento del modelo con *transfer learning* de *embeddings*

In [20]:
from gensim.models import KeyedVectors

In [21]:
MAX_WORDS = 20_000   # Número máximo de palabras para cargar y UTILIZAR el embedding
EMBEDDING_DIM = 200   # Dimensiones de los vectores de representación de palabras

In [22]:
# Cargar el modelo Stanford Glove embeddings
word2vec_file = '../../Glove/glove.twitter.27B.200d.word2vec'
glove_model = KeyedVectors.load_word2vec_format(word2vec_file, limit=MAX_WORDS)

In [23]:
# Cargar nuevamente datos de IMDB, con más palabras
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=MAX_WORDS, start_char=None)

print('Shape de los datos X de entrenamiento: ', x_train.shape)
print('Shape de los datos Y de entrenamiento: ', y_train.shape)
print('Shape de los datos X de validación: ', x_val.shape)
print('Shape de los datos Y de validación: ', y_val.shape)

Shape de los datos X de entrenamiento:  (25000,)
Shape de los datos Y de entrenamiento:  (25000,)
Shape de los datos X de validación:  (25000,)
Shape de los datos Y de validación:  (25000,)


In [25]:
# Obtener el diccionario de palabras del dataset, para convertir a los índices del embedding
#   mapea: palabra -> # índice en el dataset de keras
word_index = keras.datasets.imdb.get_word_index(path="imdb_word_index.json")

# Obtener el diccionario inverso, de índices a palabras. 
#   mapea: índice en el dataset de keras -> palabra
# index_word = {}
# for keys, index_imdb in word_index.items():
#     index_word[index_imdb] = keys

In [27]:
# Mostramos algunas entradas del diccionario
# word_index

In [28]:
# Definimos nuestra matriz de embeddings para el vocabulario
VOCAB_SIZE = len(word_index)
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM), dtype=np.float32)

# Llenar la matriz de embeddings con el modelo glove preentrenado
for word, i in word_index.items():
    try:
        # Si la palabra está en el modelo, se guarda en la fila de la matriz
        embedding_matrix[i] = glove_model[word]
    except KeyError:
        # Las palabras no encontradas en el modelo glove serán cero
        continue

print('Tamaño de la matriz de embeddings: ', embedding_matrix.shape)

Tamaño de la matriz de embeddings:  (88584, 200)


In [29]:
# Mostramos algunas entradas de la matriz de embeddings
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.4935  ,  0.35698 ,  0.66068 , ...,  0.17706 , -0.53695 ,
        -0.29699 ],
       [ 0.026598, -0.26277 ,  0.05302 , ...,  0.1305  , -0.22232 ,
         0.26772 ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [31]:
# Vamos a preprocesar los datos, en secuencias de tamaño fijo de largo 200
MAX_SEQ_LENGTH = 200
x_train_padded = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=MAX_SEQ_LENGTH)
x_val_padded = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=MAX_SEQ_LENGTH)

print('Shape de los datos X de entrenamiento: ', x_train_padded.shape)
print('Shape de los datos X de validación: ', x_val_padded.shape)

Shape de los datos X de entrenamiento:  (25000, 200)
Shape de los datos X de validación:  (25000, 200)


In [34]:
# Definición del modelo de sentimiento con transfer learning

sentiment_trlearn_model = keras.models.Sequential([
    # Capa de embeddings para trasladar cada número en las secuencias a un vector 
    # de representación de cada palabra. 
    # Notar ahora que damos los pesos iniciales (preentrenados) y los fijamos para
    # que no se modifiquen durante el entrenamiento
    keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=200, \
        embeddings_initializer = keras.initializers.Constant(embedding_matrix), \
        trainable = False),
    # Unidades LSTM para aprender los patrones en las secuencias que determinan un
    # comentario positivo o negativo
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(64),
    # Unidad de salida, clasifica si el comentario es positivo o negativo
    keras.layers.Dense(1, activation="sigmoid")
])

# Compilar el modelo 
sentiment_trlearn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar un resumen
sentiment_trlearn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 200)          17716800  
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 64)           67840     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 17,817,729
Trainable params: 100,929
Non-trainable params: 17,716,800
_________________________________________________________________


In [35]:
# Entrenamiento del modelo
history = sentiment_trlearn_model.fit(x = x_train_padded, y = y_train, \
    batch_size=32, epochs=4, \
    validation_data = (x_val_padded, y_val))

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 31s 1ms/sample - loss: 0.6261 - accuracy: 0.6332 - val_loss: 0.5274 - val_accuracy: 0.7416
Epoch 2/4
25000/25000 [==============================] - 28s 1ms/sample - loss: 0.4717 - accuracy: 0.7704 - val_loss: 0.4139 - val_accuracy: 0.8103
Epoch 3/4
25000/25000 [==============================] - 28s 1ms/sample - loss: 0.3781 - accuracy: 0.8278 - val_loss: 0.3759 - val_accuracy: 0.8347
Epoch 4/4
25000/25000 [==============================] - 28s 1ms/sample - loss: 0.3141 - accuracy: 0.8634 - val_loss: 0.3556 - val_accuracy: 0.8395


In [44]:
# Ahora vamos a utilizarlo para un ejemplo propio =P
# Después de un preprocesamiento de signos de puntuación y exclamación 
# podríamos tener algo así: 
own_reviews = ['this movie is awesome very exciting', \
    'awful movie']


In [45]:
# Mostramos el punteo de sentimiento de los reviews
for review in own_reviews: 
    print('Review: "', review, '" -> ', sentiment_review(review, sentiment_trlearn_model))

Review: " this movie is awesome very exciting " ->  0.6902058
Review: " awful movie " ->  0.39027804
